In [18]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import importlib
import own_gpt.model
importlib.reload(own_gpt.model)


from own_gpt.model import GPT
from own_gpt.utils import set_seed
from own_gpt.bpe import BPETokenizer
set_seed(3407)

In [19]:
use_own_gpt = True # use own_gpt or huggingface/transformers model?
model_type = 'gpt2-xl'
device = 'mps'

In [20]:
if use_own_gpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning

# ship model to device and set to eval mode
model.to(device)
model.eval();

number of parameters: 1557.61M
[INFO] Transferring weights from Hugging Face 'gpt2-xl' to own_gpt...
[DONE] Loaded 581 weights | Skipped 0


In [21]:
def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_own_gpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)

In [22]:
generate(prompt='Lewis Hamilton retires after', num_samples=10, steps=20)

--------------------------------------------------------------------------------
Lewis Hamilton retires after 'incident' while behind the wheel of Lotus car

Lotus driver Romain Gros
--------------------------------------------------------------------------------
Lewis Hamilton retires after his first attempt at the Monaco Grand Prix

"The car was too big for me and my
--------------------------------------------------------------------------------
Lewis Hamilton retires after a disappointing campaign

Hamilton says Mercedes did make progress, but was unable to match Red Bull

--------------------------------------------------------------------------------
Lewis Hamilton retires after the Austrian Grand Prix on Saturday 25 November 2014. Photograph: Yui Mok/PA


--------------------------------------------------------------------------------
Lewis Hamilton retires after winning in Germany

Lewis Hamilton retires after winning in Germany Sebastian Vettel finished second to Hamilton
----